In [1]:
import numpy as np 
import pandas as pd 

In [ ]:
df = pd.read_csv('single_tree/aggregated.csv')

In [ ]:
df_supplement = pd.read_csv('rset/rset.csv')

In [4]:
#rename 'balanced' column to 'balance'
df_supplement = df_supplement.rename(columns={'balanced':'balance'})

In [5]:
df = pd.concat([df, df_supplement])

In [6]:
df['proportion_nan'] = df['n_nan_preds'] / df['test_set_size']

In [7]:
df.head()

,dataset,base_tree,NaN_condition,balance,prebin_missingness,acc_complete_train,acc_complete_test,miss_prob,test_set_size,n_nan_preds,acc_incomplete_when_not_nan,acc_when_nan_if_0_impute,rset_size,avg_sparsity_complete,acc_complete_on_points_where_incomplete_not_nan,avg_sparsity_incomplete,proportion_nan
0,compas_complete.csv,gosdt,ours,False,False,0.665882,0.652677,0.1,1382,190,0.645134,0.563158,NaN,NaN,NaN,NaN,0.137482
1,compas_complete.csv,gosdt,ours,False,False,0.665882,0.652677,0.2,1382,373,0.645193,0.528150,NaN,NaN,NaN,NaN,0.269899
2,compas_complete.csv,gosdt,ours,False,False,0.665882,0.652677,0.3,1382,564,0.644254,0.526596,NaN,NaN,NaN,NaN,0.408104
3,compas_complete.csv,gosdt,ours,False,False,0.665882,0.652677,0.4,1382,700,0.636364,0.507143,NaN,NaN,NaN,NaN,0.506512
4,compas_complete.csv,gosdt,ours,False,False,0.665882,0.652677,0.5,1382,844,0.639405,0.494076,NaN,NaN,NaN,NaN,0.610709


In [8]:
groups = df.groupby(['base_tree', 'dataset', 'NaN_condition', 'miss_prob', 'balance', 'prebin_missingness'])

In [9]:
table = groups['proportion_nan'].agg(['mean', 'std', 'count'])

In [10]:
table['std_err'] = table['std']/np.sqrt(table['count'])

In [11]:
table['proportion_valid_predictions'] = 1 - table['mean']

In [12]:
printable_dataset_names = {
    'compas_complete.csv': 'COMPAS',
    'wine_quality.csv': 'Wine Quality',
    'wisconsin.csv': 'Wisconsin',
    'coupon_full.csv': 'Coupon',
}

In [13]:
datasets = ['compas_complete.csv', 'wine_quality.csv', 'wisconsin.csv', 'coupon_full.csv']
rate = 0.5
balance = False
for dataset in datasets:
    print(dataset)
    ours = table.loc[('sklearn', dataset, 'ours', rate, balance, False), 'proportion_valid_predictions']
    path = table.loc[('sklearn', dataset, 'path-based', rate, balance, False), 'proportion_valid_predictions']
    used = table.loc[('sklearn', dataset, 'used-features', rate, balance, False), 'proportion_valid_predictions']
    print(ours, table.loc[('sklearn', dataset, 'ours', rate, balance, False), 'std_err'])
    print(path, table.loc[('sklearn', dataset, 'path-based', rate, balance, False), 'std_err'])
    print(used, table.loc[('sklearn', dataset, 'used-features', rate, balance, False), 'std_err'])
    print(f'{ours/path*100:.0f}')
    print(f'{ours/used*100:.0f}')
    print('-----------------------------------------------')

compas_complete.csv
0.2995491846655719 0.01180453099030962
0.12335405770478192 0.004986102377729687
0.0091208891394583 0.0011347725062689892
243
3284
-----------------------------------------------
wine_quality.csv
0.457440990110736 0.016816382116036317
0.11682536862675441 0.003703378192971523
0.00708012080298448 0.0009546055436589492
392
6461
-----------------------------------------------
wisconsin.csv
0.35160689333954354 0.03541304264395054
0.13356621642602085 0.01451495965964367
0.012296227293898454 0.005947142974857107
263
2859
-----------------------------------------------
coupon_full.csv
0.3982082185019127 0.03657266157820503
0.11830481400737658 0.0013253423594876519
0.024753917979515427 0.0029401106192527265
337
1609
-----------------------------------------------
